<a href="https://colab.research.google.com/github/Valentin-Grachev/Machine-Learning-2023/blob/master/Student_Exam_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np              # Одномерные и многомерные массивы (array)
import pandas as pd             # Таблицы и временные ряды (dataframe, series)
import matplotlib.pyplot as plt # Научная графика
import seaborn as sns           # Еще больше красивой графики для визуализации данных
import sklearn                  # Алгоритмы машинного обучения

In [26]:
url = "https://raw.githubusercontent.com/Valentin-Grachev/Machine-Learning-2023/master/dataset.csv"
origin_data = pd.read_csv(url)

# 0. Описание задачи

## Описание датасета
Этот набор данных включает результаты трех тестов более 30 тысяч учащихся (вымышленной) государственной школы и различные личные и социально-экономические факторы, которые могут оказывать на них влияние во время взаимодействия.
## Постановка задачи
Зная социально-экономические данные студента, наиболее точно предсказать его `оценку за экзамен по математике`.

# 1. Чтение данных

In [27]:
origin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           30641 non-null  int64  
 1   Gender               30641 non-null  object 
 2   EthnicGroup          28801 non-null  object 
 3   ParentEduc           28796 non-null  object 
 4   LunchType            30641 non-null  object 
 5   TestPrep             28811 non-null  object 
 6   ParentMaritalStatus  29451 non-null  object 
 7   PracticeSport        30010 non-null  object 
 8   IsFirstChild         29737 non-null  object 
 9   NrSiblings           29069 non-null  float64
 10  TransportMeans       27507 non-null  object 
 11  WklyStudyHours       29686 non-null  object 
 12  MathScore            30641 non-null  int64  
 13  ReadingScore         30641 non-null  int64  
 14  WritingScore         30641 non-null  int64  
dtypes: float64(1), int64(4), object(10)


In [29]:
origin_data.head(3)

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,1,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,5 - 10,69,90,88
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91


## Значение признаков

*   `Gender` - пол студента (`male`, `female`)
*   `EthnicGroup` - Этническая группа студента (`group A`, `group B`, `group C`, `group D`, `group E`)
*   `ParentEduc` - Образование родителей (`some high school`, `some college`, `master's degree`, `high school`, `bachelor's degree`, `associate's degree`)
*   `LunchType` - тип школьного обеда (`standard`, `free/reduced`)
*   `TestPrep` - был ли пройден курс подготовки к тестированию (`none`, `completed`)
*   `ParentMaritalStatus` - Семейное положение родителя/родителей (`married`, `widowed`, `single`, `divorced`)
*   `PracticeSport` - как часто студент занимается спортом (`never`, `sometimes`, `regularly`)
*   `IsFirstChild` - является ли ребенок первым ребенком в семье или нет (`yes`, `no`)
*   `NrSiblings` - количество братьев и сестер у студента (`от 0 до 7`)
*   `TransportMeans` - Средства передвижения до школы (`school_bus`, `private`)
*   `WklyStudyHours` - еженедельные часы самостоятельной работы (`< 5`, `5 - 10`, `> 10`)
*   `MathScore` - тестовый балл по математике (`от 0 до 100`)
*   `ReadingScore` - тестовый балл по чтению (`от 0 до 100`)
*   `WritingScore` - оценка за письменный тест (`от 0 до 100`)

Все признаки являются категориальными, за исключением `NrSiblings` и `MathScore`, `ReadingScore`, `WritingScore` - они количественные.
Требуется предсказать количественный признак `MathScore` по остальным признакам, без использования признаков `ReadingScore`, `WritingScore`. Это задача восстановления регрессии.




















## Предобработка данных

In [59]:
# Конвертация объектных признаков в категориальные
origin_data['Gender'] = origin_data['Gender'].astype('category')
origin_data['EthnicGroup'] = origin_data['EthnicGroup'].astype('category')
origin_data['ParentEduc'] = origin_data['ParentEduc'].astype('category')
origin_data['LunchType'] = origin_data['LunchType'].astype('category')
origin_data['TestPrep'] = origin_data['TestPrep'].astype('category')
origin_data['ParentMaritalStatus'] = origin_data['ParentMaritalStatus'].astype('category')
origin_data['PracticeSport'] = origin_data['PracticeSport'].astype('category')
origin_data['IsFirstChild'] = origin_data['IsFirstChild'].astype('category')
origin_data['TransportMeans'] = origin_data['TransportMeans'].astype('category')
origin_data['WklyStudyHours'] = origin_data['WklyStudyHours'].astype('category')

# Конвертация свойства NrSiblings из float64 в int64. nan заменим на -1.
origin_data['NrSiblings'] = origin_data['NrSiblings'].fillna(-1)
origin_data['NrSiblings'] = origin_data['NrSiblings'].astype(np.int64)

# Избавимся от столбца с индексами, который не несет никакой информации
origin_data.drop(origin_data.columns[[0]], axis = 1, inplace = True)

origin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Gender               30641 non-null  category
 1   EthnicGroup          28801 non-null  category
 2   ParentEduc           28796 non-null  category
 3   LunchType            30641 non-null  category
 4   TestPrep             28811 non-null  category
 5   ParentMaritalStatus  29451 non-null  category
 6   PracticeSport        30010 non-null  category
 7   IsFirstChild         29737 non-null  category
 8   NrSiblings           30641 non-null  int64   
 9   TransportMeans       27507 non-null  category
 10  WklyStudyHours       29686 non-null  category
 11  MathScore            30641 non-null  int64   
 12  ReadingScore         30641 non-null  int64   
 13  WritingScore         30641 non-null  int64   
dtypes: category(10), int64(4)
memory usage: 1.2 MB


# 2. Визуализация данных и вычисление основных характеристик